In [260]:
import nltk
from tqdm import tqdm
import pandas as pd
from pprint import pprint
import csv

In [261]:
#specify the path to the Java-based MALLET software
path_to_mallet = 'C:/mallet-2.0.8/bin/mallet'

#import the above packages and libraries for working with files and the file system
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path

In [262]:
#assign to the "directory" variable the path to the directory containing our documents
m_directory = "C:/Users/media/Desktop/gobbykid/balanced_corpus/m"
f_directory = "C:/Users/media/Desktop/gobbykid/balanced_corpus/f"

#use `glob.gob()` function to make a list of all the `.txt` files in that directory.
files = glob.glob(f"{m_directory}/*.txt") + glob.glob(f"{f_directory}/*.txt")


Import MALLET default stopwords.

In [263]:
#import LMW stop words list (stored in the variable "STOPS"), in order to extend it
from little_mallet_wrapper.little_mallet_wrapper import STOPS as lmw_stopwords #!important: import from little_mallet_wrapper.little_mallet_wrapper, otherwise it will look for STOPS inside __init__.py (inside little_mallet_wrapper) and it will not find it!

Retrieve characters names from the characters extraction phase, in order to add them to the stopwords list. 

In [264]:
import csv
import ast
csv_path = "characters_from_full_corpus.csv"

retrieved_characters =[]
with open(csv_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        for row in csv_reader:
                retrieved_characters.extend(ast.literal_eval(row["male_characters_names"]) + ast.literal_eval(row["female_characters_names"]) + ast.literal_eval(row['unknown_gender_names']))

Define a function to split multi-word characters names 

In [265]:
def get_characters_names (characters_list):
    # empty honorifics list if you don't want to include in the model character-related words
    honorifics = []
    #honorifics = ["mr","master","mrs","miss","lady","sir","dame","lord","sister","mother","aunt","uncle","doctor","captain","father","count","professor","major","little","big","old","king","don","dr","queen","boy","girl","green","black","grey","gray","golden","silver","blue", "long"]
    output_set = set()
    #add to the previous list also the first names and last names taken by themselves (eg both "john smith" and "john" get in the list), excluding all the words in "honorifics"
    

    for item in characters_list:
        if " " in item:
            minilist = item.split()
            for w in minilist:
                if w not in honorifics:
                    output_set.add(w)
        else:
            output_set.add(item)
    
    output_list = sorted(output_set) #convert the set to a list
    return output_list

Extend the default MALLET stopwords list with characters' names and extra stopwords, to produce a custom stopwords list.

In [266]:
custom_stopwords = lmw_stopwords #initialize final list of custom stop words as containing the default stop words of lmw

more_stopwords = ["one","two","ones","say","says","said","think","thinks","thought","thing","things","go","goes","went","come","comes","came","coming","much","get","gets","got","ask","asks","asked","didn","would","could","three","other","another","until","till","upon","shall","make","made","might","must","going","way", "thou", "thee", "seems", "seem", "seemed", "never", "tell", "told", "tells", "wouldn", "tha", "like", "however","let","rather","yes", "no","little", "old", "well", "always", "never", "time", "long", "see", "saw", "sees", "illustration"]

# list of gender-related words to remove
gendered_stopwords = ['guy','dr','spokesman','chairman',"men's",'men','boy',
'boys','brother','brothers','dad','dads','dude','father',
'fathers','gentleman','gentlemen','god','grandfather','grandpa',
'grandson','groom','he','himself','his','husband','pastor','husbands','king','male','man',
'mr','nephew','nephews','priest','prince','son','sons','uncle','uncles',
'waiter','widower','widowers','lord','master',"mrs","miss","lady","sir","dame","lord","sister","mother","aunt","uncle","doctor","father","count","professor", 'heroine','drss','spokeswoman','chairwoman',"women's",'actress','women',"she's",'her','aunt','aunts','bride','daughter','daughters','female','fiancee','girl',
'girls','goddess','granddaughter','grandma','grandmother',
'herself','ladies','lady','mom','moms','mother','mothers','mrs','ms','niece',
'nieces','priestess','princess','queens','sister','sisters','waitress',
'widow','widows','wife','wives','woman','lady','mistress','queen']

characters_names = get_characters_names(retrieved_characters) #the function returns a list of strings that are (part of) a character's name

custom_stopwords.extend(more_stopwords)
custom_stopwords.extend(characters_names)
#removes words like "mr", "mrs", "mother", "father", etc.

#custom_stopwords.extend(gendered_stopwords) 

Master list of book titles in the dataset.

In [267]:
book_titles = [Path(file).stem for file in files]

**Preprocess the texts in the dataset.**

```
training_data = []

for file in tqdm(files):

    whole_text = open(file, 'r', encoding='utf-8').read() 

    # create list of pos tagged sentences of the text
    pos_tagged_sentences = [nltk.pos_tag(nltk.word_tokenize(sent)) for sent in nltk.sent_tokenize(whole_text)]
    
    # return all the nouns in the book, except custom stopwords
    nouns_in_book = []
    for l in pos_tagged_sentences:
        for (w, pos) in l:
            word = w.lower() # <-- re-define word as lowercase word!!
            if pos[0] == 'N' and word not in custom_stopwords and word.isascii() and not word.isnumeric():
                nouns_in_book.append(word)
    
    n = 1000 # number of nouns to include in each chunk
    # chunks the list of nouns for each book into x sublists with n items
    sublists_of_nouns = [nouns_in_book[i:i + n] for i in range(0, len(nouns_in_book), n)]

    chunk_pointer = 0
    for l in sublists_of_nouns:
        book_chunk = " ".join(l) # transform list to string
        training_data.append(book_chunk) #append the chunk to training data
    
        #write the book chunk to a file
        with open(f"training_data/{Path(file).stem}_chunk_{chunk_pointer}.txt", 'w', encoding='utf-8') as f:
            f.write(book_chunk)
        chunk_pointer += 1
```

Given that processing the texts is a time-consuming operation and we might want to train our model several times in order to chose the best parameters, we stored the output of the text processing in files inside training_data. We can use the following function to re-create a variable for each chunk, retrieving the ready-to-use texts directly, i.e. skipping the processing phase, the next time we use this program.

In [268]:
#re-create the training data variable from saved files

training_data = []
processed_files_paths = glob.glob("training_data/*.txt") #creates a list with the file paths of the processed texts
for processed_text in processed_files_paths:
    ready_text = open(processed_text, 'r', encoding='utf-8').read()
    training_data.append(ready_text)

In [269]:
print("Number of book chunks (documents) in the training dataset:", len(training_data))

Number of book chunks (documents) in the training dataset: 2544


## Training the topic model

We set the parameters for training the model with `little_mallet_wrapper.quick_train_topic_model()`. We must specify the path where to save the output files. Most importantly, we must choose the number of topics (`num_topics`) to be "discovered" inside our corpus: this is the only parameter we have control on in the training phase. Since there is no automatic solution to choose this parameter, trial and error seems the best strategy. We start by setting `num_topics = 40`, and later we will try increasing and decreasing the value of this parameter, to see which of these values gives the best, i.e. most interpretable, results.

In [270]:

#specify the number of topics we want returned
num_topics = 40

#set desired output directory/ies (will be created inside current directory)
output_directory_path = 'topic-model-output/all_results'

#set topic model output files! No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"



Train the model with `little_mallet_wrapper.quick_train_topic_model()`.

In [271]:
"""
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)

"""

'\nlittle_mallet_wrapper.quick_train_topic_model(path_to_mallet,\n                                             output_directory_path,\n                                             num_topics,\n                                             training_data)\n\n'

Let's train the model again, this time setting 20, 30, 50, 80 and 100 as the number of topics to get.

In [272]:
"""
num_topics = 20
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             20,
                                             training_data)


#30 topics
num_topics = 30
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)
#50 topics
num_topics = 50
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)

#80 topics
num_topics = 80
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)


#100 topics
num_topics = 100
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)

"""


'\nnum_topics = 20\nlittle_mallet_wrapper.quick_train_topic_model(path_to_mallet,\n                                             output_directory_path,\n                                             20,\n                                             training_data)\n\n\n#30 topics\nnum_topics = 30\nlittle_mallet_wrapper.quick_train_topic_model(path_to_mallet,\n                                             output_directory_path,\n                                             num_topics,\n                                             training_data)\n#50 topics\nnum_topics = 50\nlittle_mallet_wrapper.quick_train_topic_model(path_to_mallet,\n                                             output_directory_path,\n                                             num_topics,\n                                             training_data)\n\n#80 topics\nnum_topics = 80\nlittle_mallet_wrapper.quick_train_topic_model(path_to_mallet,\n                                             output_directory_path,\n          

## Analysis of the results

After having trained the model with 20, 30, 40, 50, 80 and 100 as the number of topics to return, we analyze the resulting topics. Our aim is to get the most coherent model, i.e. the model whose topics are most interpretable.

After the analysis we came up with the conclusion that the best value for `num_topics` parameter seems likely to be between 40 and 50. We decide to re-train the model **setting it to 45**. 

In [273]:
"""
#45 topics
num_topics = 45
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)
"""

'\n#45 topics\nnum_topics = 45\nlittle_mallet_wrapper.quick_train_topic_model(path_to_mallet,\n                                             output_directory_path,\n                                             num_topics,\n                                             training_data)\n'

### The 45 topics discovered in our corpus. 

In [274]:
num_topics = 45
path_to_model = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"

topic_keys = little_mallet_wrapper.load_topic_keys(path_to_topic_keys) # list of lists of strings
print(len(topic_keys))
topics_data = []

for idx, topic in enumerate(topic_keys):
    topic_dict = {}
    topic_dict['topic_index'] = idx
    topic_dict['topic_words'] = ", ".join(topic)
    topics_data.append(topic_dict)

with open('45_topic_keys.csv', 'w') as csv_file:
    
    fields = ['topic_index', 'topic_words']
    writer = csv.DictWriter(csv_file, fieldnames=fields)
    writer.writeheader()
    for row in topics_data:
        writer.writerow(row)
    

topics_df = pd.DataFrame(topics_data)
topics_df

45


,topic_index,topic_words
0,0,"face, look, voice, something, word, mind, bit, sort, eye, right, road, business, back, song, side, you, breath, arm, gentleman, book"
1,1,"men, oxford, town, rooms, year, life, work, river, university, porter, sort, fellow, play, stage, side, set, hero, rate, crew, gentlemen"
2,2,"horses, road, day, whip, men, gentleman, course, saddle, tail, legs, groom, mare, picture, children, jackanapes, tails, cab, grimes, reins, deal"
3,3,"child, day, face, children, heart, life, tears, years, arms, kind, voice, wife, days, dear, story, nothing, care, room, husband, side"
4,4,"board, men, guns, ships, vessel, shot, crew, boats, officers, deck, craft, gun, officer, vessels, pirates, frigate, shore, course, side, bay"
5,5,"school, term, house, fellows, door, right, study, meeting, side, chap, course, cheers, room, fellow, everybody, question, d'arcy, heroes, grandcourt, day"
6,6,"yer, money, street, shop, purse, face, door, bit, streets, night, policeman, court, pocket, ma'am, morning, please, wot, thief, chapter, police"
7,7,"church, men, books, sunday, soul, minister, service, book, sin, prayer, oxford, life, chapel, day, friends, work, earth, truth, sermon, others"
8,8,"boys, school, study, form, room, fellows, fellow, work, day, book, desk, holidays, class, morning, lesson, books, rest, others, house, half"
9,9,"night, door, nothing, house, something, anything, bed, wall, feet, window, day, moon, morning, work, side, moment, floor, sun, back, stones"


## The topics distribution in our corpus

To preserve significant contextual information, we have trained our topic model on the set of *chunked* the books, where each book was divided in chunks of up to 1000 tokens, and each chunk represented a document. 

Now we need to apply the model trained in this way to the very same content (the nouns in each book), but divided in units that are significant for the interpretation of the results: we need to map the topics to each of our book (not to each chunk).

In order to do so, we first pre-process the books, removing stopwords and extracting the nouns only, without chunking them. Then we apply our best model to the new training data (a list of strings where each string consists in the *whole*, cleaned, book).


Preprocess the **whole** texts, repeating the criteria chosen above, but without chunking them into equally sized sections: each document in the new model will be a string of all the nouns - separated by one whitespace character - in each of the original books. Let's also save the output of this second preprocessing phase in files inside the dedicated folder (`new_training_data`), as we did earlier for the text chunks.

```
new_training_data = []

for file in tqdm(files):

    whole_text = open(file, 'r', encoding='utf-8').read() 

    # create list of pos tagged sentences of the text
    pos_tagged_sentences = [nltk.pos_tag(nltk.word_tokenize(sent)) for sent in nltk.sent_tokenize(whole_text)]
    
    # return all the nouns in the book, except custom stopwords
    nouns_in_book_list = []
    for l in pos_tagged_sentences:
        for (w, pos) in l:
            word = w.lower() # <-- re-define word as lowercase word!!
            if pos[0] == 'N' and word not in custom_stopwords and word.isascii() and not word.isnumeric():
                nouns_in_book_list.append(word)
    
    nouns_in_book = " ".join(nouns_in_book_list) #transform list of nouns into string
    new_training_data.append(nouns_in_book) # append all the nouns in the current book to new_training_data
    
    #write the the nouns in the current book to a file
    with open(f"new_training_data/{Path(file).stem}.txt", 'w', encoding='utf-8') as f:
        f.write(nouns_in_book)
```

In [275]:
#re-create the training data variable from saved files

new_training_data = []
processed_files_paths = glob.glob("new_training_data/*.txt") #creates a list with the file paths of the processed texts
for processed_text in processed_files_paths:
    ready_text = open(processed_text, 'r', encoding='utf-8').read()
    new_training_data.append(ready_text)

Set the output paths where to store the results of the inferred model.

In [276]:
num_topics = 45 

path_to_topic_keys = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}" #topics keys are the same for both chunked texts and whole books


path_to_new_training_data = f"{output_directory_path}/results_on_whole_books/training.txt"
path_to_new_formatted_training_data = f"{output_directory_path}/results_on_whole_books/mallet.training"
new_training_data = new_training_data

path_to_new_topic_distributions = f"{output_directory_path}/results_on_whole_books/mallet.topic_distributions.{str(num_topics)}"


Import the training data to feed to the inferring function.

In [277]:
little_mallet_wrapper.import_data(path_to_mallet, path_to_new_training_data, path_to_new_formatted_training_data, new_training_data, training_ids=None, use_pipe_from=path_to_formatted_training_data)

Importing data using pipe...
Complete


Infer the new topic model: we are getting *topic distributions* for a  new set of documents (the *whole* books) using a model that has been trained on another set of documents (the books' *chunks*); nevertheless, the overall content of each of these two dataset is the same.

In [278]:
# apply the model trained on the chunked documents to the actual (whole) books
little_mallet_wrapper.infer_topics(path_to_mallet, path_to_model, path_to_new_formatted_training_data, path_to_new_topic_distributions)

Inferring topics using pre-trained model...
Complete


## Topic distributions over the corpus: results

In [279]:
topic_distributions = little_mallet_wrapper.load_topic_distributions(path_to_new_topic_distributions)

#column_names = [f"topic {topic_keys.index(l)}: " + ', '.join(l) for l in topic_keys]
column_names = [f"topic {topic_keys.index(l)}" for l in topic_keys]
topic_distributions_df = pd.DataFrame(topic_distributions, columns=column_names)


Let's import metadata of our corpus.

In [280]:
corpus_data = pd.read_csv('whole_balanced_corpus.csv')

Now merge the metadata and the topic distributions in one big single DataFrame object, in order to use it for the visualizations.

In [281]:
complete_data_df = pd.concat([corpus_data, topic_distributions_df], axis=1)
male_complete_df = complete_data_df[complete_data_df['authors_sex']=='M']
female_complete_df = complete_data_df[complete_data_df['authors_sex']=='F']


## Topic interpretation

I have tried to label as much topics as I could, based on the interpretation of the set of words in each topic and on the themes likely to be found in XIX century literature for children.

Create a dataframe with topic indexes, topic labels and topic words.

In [282]:
labelled_topics = pd.read_csv('labelled_topics.csv')
pd.set_option('display.max_colwidth', None)
labelled_topics

,Topic index,Label,Words in topic
0,2,Topic 2: Horses,"horses, road, day, whip, men, gentleman, course, saddle, tail, legs, groom, mare, picture, children, jackanapes, tails, cab, grimes, reins, deal"
1,4,Topic 4: Navy and pirates,"board, men, guns, ships, vessel, shot, crew, boats, officers, deck, craft, gun, officer, vessels, pirates, frigate, shore, course, side, bay"
2,6,Topic 6: Street life,"yer, money, street, shop, purse, face, door, bit, streets, night, policeman, court, pocket, ma'am, morning, please, wot, thief, chapter, police"
3,7,Topic 7: Church and religion,"church, men, books, sunday, soul, minister, service, book, sin, prayer, oxford, life, chapel, day, friends, work, earth, truth, sermon, others"
4,8,Topic 8: School and studying,"boys, school, study, form, room, fellows, fellow, work, day, book, desk, holidays, class, morning, lesson, books, rest, others, house, half"
5,9,Topic 9: Night and sleeping,"night, door, nothing, house, something, anything, bed, wall, feet, window, day, moon, morning, work, side, moment, floor, sun, back, stones"
6,16,Topic 16: Sea and navigation,"shore, men, island, board, vessel, boats, night, cabin, beach, sail, rocks, crew, day, oars, waves, side, fish, land, tide, work"
7,17,Topic 17: House and interior,"room, house, door, window, bed, night, morning, face, chair, moment, hour, rooms, carriage, floor, windows, stairs, servants, ladies, dinner, steps"
8,19,Topic 19: Open air,"garden, trees, birds, day, sun, grass, summer, air, leaves, fairies, wings, nothing, earth, branches, morning, sky, ground, spring, feet, side"
9,20,Topic 20: Indian Americans and life in the wilderness,"indians, men, river, lake, night, ice, camp, bear, village, trail, canoe, moonlight, feet, prairie, day, hunting, wolves, tribe, deer, friends"


In [283]:
topics_with_label = ["topic 2", "topic 4", "topic 6", "topic 7", "topic 8", "topic 9", "topic 16", "topic 17", "topic 19", "topic 20", "topic 25", "topic 27", "topic 31", "topic 32", "topic 35", "topic 39", "topic 40", "topic 44"]
new_topic_labels = list(labelled_topics['Label'])

new_columns = {} # dictionary to rename the colums of the labelled topics in the dataframe

for e in new_topic_labels:
    var = (e.split(':')[0]).lower().strip()
    for i in topics_with_label:
        if i == var:
            new_columns[i] = e

    

Rename the column names corresponding to the labelled topics in the dataframe.

In [284]:
complete_data_df.rename(columns = new_columns, inplace = True)
male_complete_df.rename(columns = new_columns, inplace = True)
female_complete_df.rename(columns = new_columns, inplace = True)

complete_data_df

C:\Users\media\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,book_title,author,year,decade,tokens_in_book,authors_sex,topic 0,topic 1,Topic 2: Horses,topic 3,...,Topic 35: Prison and criminality,topic 36,topic 37,topic 38,Topic 39: Battles and Nations,Topic 40: Coal mines and fire,topic 41,topic 42,topic 43,Topic 44: Nature and native People
0,1835_sherwood-shanty-the-blacksmith-a-tale-of-other-times,sherwood,1835,1830s,31953,F,0.013208,0.000011,0.000010,0.076550,...,0.013037,0.000136,0.027409,0.014346,0.014548,0.000010,1.836163e-06,0.000153,0.092480,0.027058
1,1836_marryat-mr-midshipman-easy,marryat,1836,1830s,139069,M,0.005296,0.000002,0.000002,0.018321,...,0.021688,0.000001,0.000169,0.039978,0.002114,0.000078,4.121288e-07,0.000052,0.125925,0.000651
2,1839_sinclair-holiday-house-a-series-of-tales,sinclair,1839,1830s,77184,F,0.004114,0.004441,0.010250,0.067933,...,0.005991,0.000002,0.000039,0.035725,0.000121,0.003219,7.176720e-07,0.000212,0.063364,0.012356
3,1841_marryat-masterman-ready,marryat,1841,1840s,99247,M,0.000788,0.000004,0.000013,0.036165,...,0.000321,0.000076,0.000027,0.022415,0.000017,0.000816,6.511292e-07,0.000008,0.069019,0.173803
4,1841_martineau-the-settlers-at-home,martineau,1841,1840s,55796,F,0.020596,0.000008,0.000007,0.071338,...,0.000050,0.000022,0.000034,0.083435,0.004714,0.000025,1.265030e-06,0.000087,0.014985,0.062152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,1912_collingwood-two-gallant-sons-of-davon-a-tale-of-the-days-of-queen-bess,collingwood,1912,1910s,108321,M,0.000133,0.000003,0.000002,0.000705,...,0.032713,0.000008,0.004355,0.065036,0.011257,0.000002,4.619529e-07,0.001080,0.014975,0.166625
262,1912_nesbit-the-magic-world,nesbit,1912,1910s,58503,F,0.043854,0.000049,0.007362,0.042401,...,0.000199,0.207204,0.000013,0.019473,0.026985,0.004246,1.017385e-06,0.014206,0.027817,0.001592
263,1912_potter-the-tale-of-mr-tod,potter,1912,1910s,4886,F,0.001680,0.000057,0.000051,0.006879,...,0.000790,0.000030,0.000119,0.047598,0.000661,0.000052,9.590108e-06,0.000391,0.003063,0.006580
264,1913_nesbit-wet-magic,nesbit,1913,1910s,48960,F,0.115496,0.000007,0.138845,0.029643,...,0.013705,0.188022,0.000048,0.028146,0.044166,0.000006,1.026419e-03,0.000858,0.002704,0.006489


Let's drop the columns we are not interested in, i.e. the ones we are not able to label.

In [285]:
columns_to_drop = []#list(complete_data_df.columns) - topics_with_label + ['book_title','author','year','decade','tokens_in_book','authors_sex']

for i in list(complete_data_df.columns):
    if i not in ['book_title','author','year','decade','tokens_in_book','authors_sex']:
        if i not in new_topic_labels:
            columns_to_drop.append(i)

for c in columns_to_drop:
    complete_data_df.drop(c, axis=1, inplace=True)
    male_complete_df.drop(c, axis=1, inplace=True)
    female_complete_df.drop(c, axis=1, inplace=True)

C:\Users\media\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [286]:
complete_data_df

,book_title,author,year,decade,tokens_in_book,authors_sex,Topic 2: Horses,Topic 4: Navy and pirates,Topic 6: Street life,Topic 7: Church and religion,...,Topic 19: Open air,Topic 20: Indian Americans and life in the wilderness,Topic 25: Feelings and sentiments,Topic 27: Nation,Topic 31: Armed conflict,Topic 32: Landscape: village,Topic 35: Prison and criminality,Topic 39: Battles and Nations,Topic 40: Coal mines and fire,Topic 44: Nature and native People
0,1835_sherwood-shanty-the-blacksmith-a-tale-of-other-times,sherwood,1835,1830s,31953,F,0.000010,0.000070,0.000354,0.025197,...,0.009879,0.000006,2.929400e-06,0.004821,0.000308,0.130056,0.013037,0.014548,0.000010,0.027058
1,1836_marryat-mr-midshipman-easy,marryat,1836,1830s,139069,M,0.000002,0.169690,0.000120,0.006929,...,0.000143,0.000001,6.575070e-07,0.015462,0.015015,0.002711,0.021688,0.002114,0.000078,0.000651
2,1839_sinclair-holiday-house-a-series-of-tales,sinclair,1839,1830s,77184,F,0.010250,0.005104,0.007362,0.010123,...,0.028901,0.000002,3.287610e-03,0.005415,0.007416,0.004863,0.005991,0.000121,0.003219,0.012356
3,1841_marryat-masterman-ready,marryat,1841,1840s,99247,M,0.000013,0.013401,0.000005,0.000064,...,0.009919,0.000002,1.038806e-06,0.006971,0.003802,0.003018,0.000321,0.000017,0.000816,0.173803
4,1841_martineau-the-settlers-at-home,martineau,1841,1840s,55796,F,0.000007,0.000030,0.000010,0.003317,...,0.049653,0.000004,2.018219e-06,0.000039,0.000427,0.176139,0.000050,0.004714,0.000025,0.062152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,1912_collingwood-two-gallant-sons-of-davon-a-tale-of-the-days-of-queen-bess,collingwood,1912,1910s,108321,M,0.000002,0.035004,0.002473,0.009627,...,0.006461,0.029277,7.369960e-07,0.008758,0.038214,0.010911,0.032713,0.011257,0.000002,0.166625
262,1912_nesbit-the-magic-world,nesbit,1912,1910s,58503,F,0.007362,0.000168,0.003860,0.011048,...,0.068362,0.003898,5.931914e-05,0.002397,0.000099,0.011192,0.000199,0.026985,0.004246,0.001592
263,1912_potter-the-tale-of-mr-tod,potter,1912,1910s,4886,F,0.000051,0.001043,0.000079,0.000397,...,0.019917,0.000165,1.512640e-04,0.000297,0.000385,0.001624,0.000790,0.000661,0.000052,0.006580
264,1913_nesbit-wet-magic,nesbit,1913,1910s,48960,F,0.138845,0.000011,0.001647,0.000032,...,0.024316,0.000070,1.860828e-06,0.011645,0.045555,0.000082,0.013705,0.044166,0.000006,0.006489


Let's pivot the table, in order to get topics as rows.

In [287]:
pivot_df = complete_data_df.pivot(columns='authors_sex', index=new_topic_labels)

## Visualization

In [288]:
import plotly.express as px
import chart_studio
import chart_studio.plotly as csp

In [289]:
username = 'eliarizzetto' # your username
api_key = 'pPHUDola2xKGXkXvr0sq' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

### The labelled topics in the whole corpus across time

In [290]:
topics_to_visualize = new_topic_labels


fig = px.bar(
    complete_data_df, 
    x="year", 
    y=topics_to_visualize,
    title="The labelled topics in the whole corpus across time",
    hover_name='book_title',
    hover_data=['authors_sex']
    )
fig.show()

print(csp.plot(fig, filename = "The labelled topics in the whole corpus across time", auto_open=False)) #create a file on plotly chart studio account and a link in this notebook

https://plotly.com/~eliarizzetto/82/


### Frequency distribution of labelled topics in female authors

In [291]:
topics_to_visualize = new_topic_labels


fig = px.histogram(
    female_complete_df, 
    x='decade', 
    y=topics_to_visualize,
    #size=topics_to_visualize,
    barmode='group',
    title="Frequency distribution of labelled topics in female authors",
    hover_data=['year'],
    )
fig.show()

print(csp.plot(fig, filename = "Frequency distribution of labelled topics in female authors", auto_open=False)) #create a file on plotly chart studio account and a link in this notebook

https://plotly.com/~eliarizzetto/90/


### Frequency distribution of labelled topics in male authors

In [292]:
topics_to_visualize = new_topic_labels


fig = px.histogram(
    male_complete_df, 
    x='decade', 
    y=topics_to_visualize,
    #size=topics_to_visualize,
    barmode='group',
    title="Frequency distribution of labelled topics in male authors",
    hover_data=['year'],
    )
fig.show()

print(csp.plot(fig, filename = "Frequency distribution of labelled topics in male authors", auto_open=False)) #create a file on plotly chart studio account and a link in this notebook

https://plotly.com/~eliarizzetto/92/


#### Topic 7: Church and religion

In [293]:
from matplotlib.pyplot import bar


topic_to_visualize = 'Topic 7: Church and religion'

fig = px.histogram(
    complete_data_df,
    x='year',
    y=f"{topic_to_visualize}",
    color='authors_sex',
    #markers=True,
    hover_name='book_title',
    labels={},
    barmode='group',
    title='Topic 7: Church and religion',
    color_discrete_sequence=['MediumSeaGreen', 'Tomato'],
)
fig.show()
print(csp.plot(fig, filename = 'Topic 7: Church and religion', auto_open=False)) #create a file on plotly chart studio account and a link in this notebook


https://plotly.com/~eliarizzetto/94/


In [294]:
topic_to_visualize = 'Topic 25: Feelings and sentiments'

fig = px.histogram(
    complete_data_df,
    x='year',
    y=f"{topic_to_visualize}",
    color='authors_sex',
    #markers=True,
    hover_name='book_title',
    labels={},
    barmode='group',
    title="Topic 25: Feelings and sentiments",
    color_discrete_sequence=['MediumSeaGreen', 'Tomato'],
)
fig.show()
print(csp.plot(fig, filename = 'Topic 25: Feelings and sentiments', auto_open=False)) #create a file on plotly chart studio account and a link in this notebook


https://plotly.com/~eliarizzetto/97/


In [295]:
topic_to_visualize = 'Topic 39: Battles and Nations'

fig = px.bar(
    complete_data_df,
    x='decade',
    y=topic_to_visualize,
    color='authors_sex',
    barmode='group',
    hover_data=['book_title'],
    title='Topic 39: Battles and Nations',
    color_discrete_sequence=['MediumSeaGreen', 'Tomato'],
)
fig.show()



print(csp.plot(fig, filename = 'Topic 39: Battles and Nations', auto_open=False)) #create a file on plotly chart studio account and a link in this notebook


https://plotly.com/~eliarizzetto/99/


In [296]:
topic_to_visualize = 'Topic 4: Navy and pirates'

fig = px.bar(
    complete_data_df, 
    x="decade", 
    y=topic_to_visualize,
    color='authors_sex', 
    barmode='group',
    hover_name='book_title',
    height=400,
    color_discrete_sequence=['MediumSeaGreen', 'Tomato'],
    title='Topic 4: Navy and pirates',
    )
fig.show()

print(csp.plot(fig, filename = 'Topic 4: Navy and pirates', auto_open=False)) #create a file on plotly chart studio account and a link in this notebook


https://plotly.com/~eliarizzetto/105/


In [297]:
# indors vs outodoor

fig = px.scatter(
    complete_data_df, 
    x='Topic 17: House and interior', 
    y='Topic 19: Open air',
    color='year', 
    hover_name='book_title',
    hover_data=['authors_sex'],
    title="Topic 17: House and interior vs Topic 19: Open air",
    #symbol='authors_sex'
    facet_col='authors_sex'
    )
fig.show()

print(csp.plot(fig, filename = "Topic 17: House and interior vs Topic 19: Open air", auto_open=False)) #create a file on plotly chart studio account and a link in this notebook


https://plotly.com/~eliarizzetto/59/
